# Programming Assignment 3 - Simple Linear versus Ridge Regression 

Your friend Bob just moved to Boston. He is a real estate agent who is trying to evaluate the prices of houses in the Boston area. He has been using a linear regression model but he wonders if he can improve his accuracy on predicting the prices for new houses. He comes to you for help as he knows that you're an expert in machine learning. 

As a pro, you suggest doing a *polynomial transformation*  to create a more flexible model, and performing ridge regression since having so many features compared to data points increases the variance. 

Bob, however, being a skeptic isn't convinced. He wants you to write a program that illustrates the difference in training and test costs for both linear and ridge regression on the same dataset. Being a good friend, you oblige and hence this assignment :) 

In this notebook, you are to explore the effects of ridge regression.  We will use a dataset that is part of the sklearn.dataset package.  Learn more at https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

## Step 1:  Getting, understanding, and preprocessing the dataset

We first import the standard libaries and some libraries that will help us scale the data and perform some "feature engineering" by transforming the data into $\Phi_2({\bf x})$

In [1]:
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold

###  Importing the dataset

In [2]:
# Import the boston dataset from sklearn
boston_data = load_boston()

In [3]:
#  Create X and Y variables - X holding the .data and Y holding .target 
X = boston_data.data
y = boston_data.target

#  Reshape Y to be a rank 2 matrix 
y = y.reshape(X.shape[0], 1)

# Observe the number of features and the number of labels
print('The number of features is: ', X.shape[1])
# Printing out the features
print('The features: ', boston_data.feature_names)
# The number of examples
print('The number of exampels in our dataset: ', X.shape[0])
#Observing the first 2 rows of the data
print(X[0:2])


The number of features is:  13
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
The number of exampels in our dataset:  506
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]


We will also create polynomial feeatures for the dataset to test linear and ridge regression on data with d = 1 and data with d = 2. Feel free to increase the # of degress and see what effect it has on the training and test error. 

In [4]:
# Create a PolynomialFeatures object with degree = 2. 
# Transform X and save it into X_2. Simply copy Y into Y_2 
poly = PolynomialFeatures(degree=2)
X_2 = poly.fit_transform(X)
y_2 = y

In [5]:
# the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
print(X_2.shape)
print(y_2.shape)

(506, 105)
(506, 1)


# Your code goes here

In [6]:
# TODO - Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# TODO - Return w values

def get_coeff_ridge_normaleq(X_train, y_train, alpha):
    # use np.linalg.pinv(a)
    #### TO-DO #####
    if alpha == 0:
        w = np.dot(np.linalg.pinv(X_train), y_train)
    else:
        I = np.eye(X_train.shape[1])
        inv = np.dot(X_train.T, X_train) + alpha*I
        inv = np.linalg.pinv(inv)
        w = np.dot(inv, X_train.T)
        w = np.dot(w, y_train)
    ##############
    return w

In [28]:
# TODO - Define the evaluate_err_ridge function.
# TODO - Return the train_error and test_error values
def evaluate_err(X_train, X_test, y_train, y_test, w): 
    #### TO-DO #####
    y_train_pre = np.dot(X_train, w)
    y_test_pre = np.dot(X_test, w)
    
    train_rss = np.sum((y_train_pre-y_train)**2)
    test_rss = np.sum((y_test_pre-y_test)**2)
    

    train_error = train_rss / len(X_train)
    test_error = test_rss / len(X_test)
    ##############
    return train_error, test_error

In [29]:
# TODO - Finish writting the k_fold_cross_validation function. 
# TODO - Returns the average training error and average test error from the k-fold cross validation
# use Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold_cross_validation(k, X, y, alpha):
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
        y_train_mean = np.mean(y_train)
        y_train = y_train - y_train_mean
        y_test = y_test - y_train_mean
        # scaling the data matrix
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # determine the training error and the test error
        #### TO-DO #####
        w = get_coeff_ridge_normaleq(X_train, y_train, alpha)
        
        train_error, test_error = evaluate_err(X_train, X_test, y_train, y_test, w)
        total_E_val_test += test_error
        total_E_val_train += train_error
        
    
       ##############
    E_val_test = total_E_val_test / k
    E_val_train = total_E_val_train / k
    return  E_val_test, E_val_train
    


In [30]:
E_val_test, E_val_train = k_fold_cross_validation(10, X, y, 0.0)
print("Linear regression model, no polymoial transformation")
print("train score: {}".format(E_val_train))
print("test score: {}".format(E_val_test))

print("*****************")
alpha = np.logspace(1, 7, num=13)
print("Ridge regression model, no polymoial transformation")
for a in alpha:
    E_val_test, E_val_train = k_fold_cross_validation(10, X, y, a)
    print("When lamda is {}, train score: {} | test score: {}".format(a, E_val_train, E_val_test))
    print("=============")

Linear regression model, no polymoial transformation
train score: 21.806183575851065
test score: 23.63606860542818
*****************
Ridge regression model, no polymoial transformation
When lamda is 10.0, train score: 21.892901157570186 | test score: 23.68858300163874
When lamda is 31.622776601683793, train score: 22.2854440556975 | test score: 24.01784029738614
When lamda is 100.0, train score: 23.725488384882077 | test score: 25.293852553695135
When lamda is 316.22776601683796, train score: 28.166554098540125 | test score: 29.457296222896804
When lamda is 1000.0, train score: 38.53214872734357 | test score: 39.48949335939871
When lamda is 3162.2776601683795, train score: 54.0070264995545 | test score: 54.6471910779495
When lamda is 10000.0, train score: 69.2597817132532 | test score: 69.7185175088451
When lamda is 31622.776601683792, train score: 78.53074243629268 | test score: 78.92162110435103
When lamda is 100000.0, train score: 82.40325012491367 | test score: 82.7724042894599
Whe

In [31]:
E_val_test, E_val_train = k_fold_cross_validation(10, X_2, y_2, 0.0)
print("Linear regression model, with polymoial transformation")
print("train score: {}".format(E_val_train))
print("test score: {}".format(E_val_test))

print("*****************")
alpha = np.logspace(1, 7, num=13)
print("Ridge regression model, with polymoial transformation")
for a in alpha:
    E_val_test, E_val_train = k_fold_cross_validation(10, X_2, y_2, a)
    print("When lamda is {}, train score: {} | test score: {}".format(a, E_val_train, E_val_test))
    print("=============")

Linear regression model, with polymoial transformation
train score: 5.808820816012461
test score: 11.854968235566691
*****************
Ridge regression model, with polymoial transformation
When lamda is 10.0, train score: 10.049055874118883 | test score: 13.476138001135917
When lamda is 31.622776601683793, train score: 12.751706269046775 | test score: 15.82960196905122
When lamda is 100.0, train score: 16.222690593210803 | test score: 18.980018815009977
When lamda is 316.22776601683796, train score: 19.700253646409852 | test score: 22.068692308062715
When lamda is 1000.0, train score: 24.287457983108887 | test score: 26.21847559440484
When lamda is 3162.2776601683795, train score: 33.08666831985053 | test score: 34.580266018800174
When lamda is 10000.0, train score: 46.761999359414084 | test score: 47.789520587567495
When lamda is 31622.776601683792, train score: 62.00900191683604 | test score: 62.646288813443356
When lamda is 100000.0, train score: 74.28758894586741 | test score: 74.7

In [34]:
x_pre = np.array([5, 0.5, 2, 0, 4, 8, 4, 6, 2, 2, 2, 4, 5.5])
x_pre = x_pre.reshape(1, 13)
x_pre_2 = poly.fit_transform(x_pre)

X_train = X_2
y_train = y_2
# centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
y_train_mean = np.mean(y_train)
y_train = y_train - y_train_mean

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

w = get_coeff_ridge_normaleq(X_train, y_train, 0.0)
y_pre = np.dot(x_pre_2, w) + y_train_mean
print("The predict price should be: {}".format(y_pre[0]))

The price should be: [21.89514111]
